* [LightGBM BayesOpt](https://www.kaggle.com/sz8416/simple-bayesian-optimization-for-lightgbm)
* [LightGBM GPU 사용 방법 1](http://www.kwangsiklee.com/2018/05/lightgbm-%EC%82%AC%EC%9A%A9%EC%8B%9C-gpu-%EA%B0%80%EC%86%8D%ED%95%98%EA%B8%B0/)
* [LightGBM GPU 사용 방법 2](https://github.com/Microsoft/LightGBM/blob/master/docs/GPU-Tutorial.rst)
* [LightGBM Docs](https://lightgbm.readthedocs.io/en/latest/)

In [5]:
# !pip install bayesian-optimization --user

import pandas as pd , numpy as np
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from collections import Counter
from IPython.display import clear_output
import collections, os
import pickle
from sklearn.preprocessing import normalize
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    result = 'Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    )
    print(result)
    f = open("lightgbm_acc.txt",'a')
    f.write(result)
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [3]:
df = pd.read_csv("./Data/dont-overfit-ii/train.csv")
df.shape

test_df = pd.read_csv("./Data/dont-overfit-ii/test.csv")
test_df = test_df.drop("id", axis = 1)
label = df["target"]
Input = df.drop(columns=['id', 'target'])

test_df = test_df.values.astype(np.float32)
x_data = Input.values.astype(np.float32)

x_data = normalize(x_data)
test_df = normalize(test_df)


y_datalabel = label
y_data = LabelEncoder().fit_transform(label)

onehot = np.zeros((y_data.shape[0], np.unique(y_data).shape[0]))
for i in range(y_data.shape[0]):
    onehot[i, y_data[i]] = 1.0

#x_train, x_test, y_train, y_test, y_train_label, y_test_label = train_test_split(x_data, onehot, y_data, test_size = 0.3)


In [ ]:
bayes_cv_tuner = BayesSearchCV(
    estimator = lgb.LGBMRegressor(
        objective='binary',
        metric='auc',
        n_jobs=10 ,
        seed = 10 ,
        device = "gpu" ,
        gpu_platform_id = 2 ,
        gpu_device_id = 2
    ),
    search_spaces = {
        'learning_rate': (0.001, 0.01, 'log-uniform'),
        'num_leaves': (5, 50),      
        'max_depth': (4, 9),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample': (0.01, 1.0, 'uniform'),
        'subsample_freq': (0, 10),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'min_child_weight': (0, 10),
        'subsample_for_bin': (100000, 500000),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 500, 'log-uniform'),
        'n_estimators': (50, 300),
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits= 10 ,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = 10000 ,   
    verbose = 0,
    refit = True,
    random_state = 42
)

# Fit the model
result = bayes_cv_tuner.fit(x_data , y_datalabel.values, callback=status_print)

Model #1
Best ROC-AUC: 0.5
Best params: {'scale_pos_weight': 0.060830282487222144, 'colsample_bytree': 0.4160029192647807, 'max_bin': 940, 'subsample': 0.13556548021189216, 'max_depth': 6, 'min_child_samples': 34, 'subsample_for_bin': 171234, 'reg_lambda': 0.05936070635912049, 'min_child_weight': 4, 'reg_alpha': 5.497557739289786e-07, 'num_leaves': 38, 'learning_rate': 0.0053422688874711095, 'n_estimators': 138, 'subsample_freq': 6}

Model #2
Best ROC-AUC: 0.616
Best params: {'scale_pos_weight': 0.3016410771843142, 'colsample_bytree': 0.8390144719977516, 'max_bin': 373, 'subsample': 0.9923710598637134, 'max_depth': 9, 'min_child_samples': 43, 'subsample_for_bin': 406716, 'reg_lambda': 276.5424475574225, 'min_child_weight': 1, 'reg_alpha': 0.0005266983003701547, 'num_leaves': 21, 'learning_rate': 0.007643904937766527, 'n_estimators': 85, 'subsample_freq': 4}

Model #3
Best ROC-AUC: 0.7313
Best params: {'scale_pos_weight': 1.3165669602830552, 'colsample_bytree': 0.4503841871781403, 'max_

/home/gpuadmin/.local/lib/python3.5/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Model #20
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.539275280492818, 'max_depth': 5, 'min_child_samples': 31, 'subsample_for_bin': 139949, 'reg_lambda': 0.2664055429156255, 'min_child_weight': 7, 'reg_alpha': 1.0, 'num_leaves': 40, 'learning_rate': 0.009867384287118095, 'n_estimators': 247, 'subsample_freq': 7}



/home/gpuadmin/.local/lib/python3.5/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Model #21
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.539275280492818, 'max_depth': 5, 'min_child_samples': 31, 'subsample_for_bin': 139949, 'reg_lambda': 0.2664055429156255, 'min_child_weight': 7, 'reg_alpha': 1.0, 'num_leaves': 40, 'learning_rate': 0.009867384287118095, 'n_estimators': 247, 'subsample_freq': 7}

Model #22
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.539275280492818, 'max_depth': 5, 'min_child_samples': 31, 'subsample_for_bin': 139949, 'reg_lambda': 0.2664055429156255, 'min_child_weight': 7, 'reg_alpha': 1.0, 'num_leaves': 40, 'learning_rate': 0.009867384287118095, 'n_estimators': 247, 'subsample_freq': 7}

Model #23
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.53927528049281

...

Model #49
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.539275280492818, 'max_depth': 5, 'min_child_samples': 31, 'subsample_for_bin': 139949, 'reg_lambda': 0.2664055429156255, 'min_child_weight': 7, 'reg_alpha': 1.0, 'num_leaves': 40, 'learning_rate': 0.009867384287118095, 'n_estimators': 247, 'subsample_freq': 7}

Model #50
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.539275280492818, 'max_depth': 5, 'min_child_samples': 31, 'subsample_for_bin': 139949, 'reg_lambda': 0.2664055429156255, 'min_child_weight': 7, 'reg_alpha': 1.0, 'num_leaves': 40, 'learning_rate': 0.009867384287118095, 'n_estimators': 247, 'subsample_freq': 7}

Model #51
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.53927528049281

/home/gpuadmin/.local/lib/python3.5/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Model #54
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.539275280492818, 'max_depth': 5, 'min_child_samples': 31, 'subsample_for_bin': 139949, 'reg_lambda': 0.2664055429156255, 'min_child_weight': 7, 'reg_alpha': 1.0, 'num_leaves': 40, 'learning_rate': 0.009867384287118095, 'n_estimators': 247, 'subsample_freq': 7}

Model #55
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.539275280492818, 'max_depth': 5, 'min_child_samples': 31, 'subsample_for_bin': 139949, 'reg_lambda': 0.2664055429156255, 'min_child_weight': 7, 'reg_alpha': 1.0, 'num_leaves': 40, 'learning_rate': 0.009867384287118095, 'n_estimators': 247, 'subsample_freq': 7}

Model #56
Best ROC-AUC: 0.784
Best params: {'scale_pos_weight': 8.204911568371568, 'colsample_bytree': 0.7011941273318365, 'max_bin': 100, 'subsample': 0.53927528049281

/home/gpuadmin/.local/lib/python3.5/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Model #64
Best ROC-AUC: 0.7979
Best params: {'scale_pos_weight': 1.2302384486290132, 'colsample_bytree': 0.6620405130534834, 'subsample': 0.6230031593324037, 'max_bin': 100, 'max_depth': 4, 'min_child_samples': 35, 'subsample_for_bin': 100000, 'reg_lambda': 0.00046521830526982015, 'min_child_weight': 8, 'reg_alpha': 0.048229854873648295, 'num_leaves': 29, 'learning_rate': 0.006903161354556259, 'n_estimators': 300, 'subsample_freq': 10}

Model #65
Best ROC-AUC: 0.7979
Best params: {'scale_pos_weight': 1.2302384486290132, 'colsample_bytree': 0.6620405130534834, 'subsample': 0.6230031593324037, 'max_bin': 100, 'max_depth': 4, 'min_child_samples': 35, 'subsample_for_bin': 100000, 'reg_lambda': 0.00046521830526982015, 'min_child_weight': 8, 'reg_alpha': 0.048229854873648295, 'num_leaves': 29, 'learning_rate': 0.006903161354556259, 'n_estimators': 300, 'subsample_freq': 10}

Model #66
Best ROC-AUC: 0.7979
Best params: {'scale_pos_weight': 1.2302384486290132, 'colsample_bytree': 0.66204051305